In [1]:
import torch
import json
from tvm import autotvm

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
import sys
import numpy as np
import tvm
from tvm import autotvm
import tvm.te as te
import tvm.testing
from masker_scheduler import *

gpu_id=1
os.environ['CUDA_VISIBLE_DEVICES'] = f'{gpu_id}'
refresh=0 # clear prev search
target="cuda"
device=tvm.device(target, 0)
task_name="dyconv/masker"

# widths, channels, group_width = regnet_parameters("008")
# regnet 008
widths = [56, 28, 14, 7]
channels = [64, 128, 288, 672]
group_width = 16
batch_size=1

test_densities = [0.2, 0.4, 0.6, 0.8]

for density in test_densities:
    for i in range(4):
        width, channel = widths[i], channels[i]
        height=width
        n_trial = 300
        eval_repeat=100
        granul_groups=1
        save_dir = f"log/c{channel}_w{width}_g{group_width}"
        
        granul_sizes = get_factors(width)
        for granul_size in granul_sizes:
            n_blocks = find_best_n_blocks(width//granul_size, density-0.01, density+0.01)
            save_path=f"{save_dir}/maskconv.json"
            args=[batch_size,granul_groups,channel,width,height,granul_size]
            task = autotvm.task.create(
                task_name=task_name, args=args, target=target
            ) # Create a tuning task and initialize its search space
            with autotvm.apply_history_best(save_path):
                with tvm.target.Target(target):
                    s, arg_bufs = masker_conv(*args)
                    func = tvm.build(s, arg_bufs)
                    
                    evaluator=func.time_evaluator(func.entry_name,device,eval_repeat)
                    input = np.random.randn(batch_size, channel, height, width).astype("float32")
                    weight = np.random.randn(granul_groups, channel).astype("float32")
                    output = np.zeros([batch_size,granul_groups,height//granul_size,width//granul_size]).astype("float32")
                    sample=[input, weight,output]
                    sample = [tvm.nd.array(_, device) for _ in sample]
                    t=evaluator(*sample).mean
                    with open(save_path+'.best','w') as f:
                        f.write(f"time {t} \n")
                        f.write(f"{tvm.lower(s, arg_bufs, simple_mode=True)}")
                        
            # measure_option = autotvm.measure_option(
            #     builder=autotvm.LocalBuilder(),
            #     runner=autotvm.LocalRunner(repeat=3, min_repeat_ms=150, timeout=4),
            # )
            # tuner = autotvm.tuner.XGBTuner(task)
            # tuner.tune(
            #     n_trial=n_trial,
            #     measure_option=measure_option,
            #     callbacks=[autotvm.callback.log_to_file(save_path)],
            # )
            print(f"autotune {save_path} complete!")

Cannot find config for target=cuda -keys=cuda,gpu -arch=sm_70 -max_num_threads=1024 -thread_warp_size=32, workload=('dyconv/masker', 1, 1, 64, 56, 56, 2). A fallback configuration is used, which may bring great performance regression.


autotune log/c64_w56_g16/maskconv.json complete!


Cannot find config for target=cuda -keys=cuda,gpu -arch=sm_70 -max_num_threads=1024 -thread_warp_size=32, workload=('dyconv/masker', 1, 1, 64, 56, 56, 4). A fallback configuration is used, which may bring great performance regression.


autotune log/c64_w56_g16/maskconv.json complete!
autotune log/c64_w56_g16/maskconv.json complete!


Cannot find config for target=cuda -keys=cuda,gpu -arch=sm_70 -max_num_threads=1024 -thread_warp_size=32, workload=('dyconv/masker', 1, 1, 64, 56, 56, 8). A fallback configuration is used, which may bring great performance regression.


...

Cannot find config for target=cuda -keys=cuda,gpu -arch=sm_70 -max_num_threads=1024 -thread_warp_size=32, workload=('dyconv/masker', 1, 1, 64, 56, 56, 14). A fallback configuration is used, which may bring great performance regression.


autotune log/c64_w56_g16/maskconv.json complete!


Cannot find config for target=cuda -keys=cuda,gpu -arch=sm_70 -max_num_threads=1024 -thread_warp_size=32, workload=('dyconv/masker', 1, 1, 64, 56, 56, 28). A fallback configuration is used, which may bring great performance regression.


autotune log/c64_w56_g16/maskconv.json complete!
autotune log/c64_w56_g16/maskconv.json complete!


FileNotFoundError: [Errno 2] No such file or directory: 'log/c128_w28_g16/maskconv.json'